Objective: The purpose of this notebook is to extracxt comorbidities of our cohort based on which we have binary indicators of a comorbidity being present within the given time frame before specimen collection. This is the Step2 of this process.

# pre_requsits

In [ ]:
!pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage

In [ ]:
!pip list | grep google-cloud-bigquery

In [ ]:
!python3 -m venv env_name
!source env_name/bin/activate
!pip install google-cloud-bigquery google-cloud-bigquery-storage
from google.cloud import bigquery_storage_v1

In [ ]:
!pip install db-dtypes

In [ ]:
!pip install pandas-gbq

# Lib

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/fa/Downloads/google-cloud-sdk/som-nero-phi-jonc101-2116f9f7bc68.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

In [17]:
import pdb
import re
unique_values=set()

for column in ['present_component','component_7days','component_14days','component_30days','component_90days','component_180days']:
    query = f"""
    SELECT DISTINCT {column}
    FROM `som-nero-phi-jonc101.antimicrobial_stewardship.microbiology_comorbidity_temp`
    where {column} is not null
    """
    results = client.query(query).result()
    unique_values.update([row[column] for row in results])

def fix(value):
    value=re.sub(r'\s{1,}', '_', value).strip()
    value = re.sub(r'[^\w\s_]', '', value)
    return value
    
All_Components=[value for value in unique_values]
select_clause = "*"
for column in ['present_component','component_7days','component_14days','component_30days','component_90days','component_180days']:
    for value in All_Components:
        select_clause += f", CASE WHEN present_component = '{value}' THEN 1 ELSE 0 END AS {fix(value)}_{column}"
        
final_query = f"""
CREATE OR REPLACE TABLE `som-nero-phi-jonc101.antimicrobial_stewardship.microbiology_comorbidity` AS(
SELECT {select_clause}
FROM `som-nero-phi-jonc101.antimicrobial_stewardship.microbiology_comorbidity`)
"""


In [18]:
results = client.query(final_query).result()

In [3]:
#AHRQ_CCSR_df['AHRQ_CCSR_CATEGORY_Description_R']=AHRQ_CCSR_df['AHRQ_CCSR_CATEGORY_Description'].apply(lambda x:'_'.join(x.split(' '))) # need to remove special chars

In [6]:
#AHRQ_CCSR_df.drop(columns=['AHRQ_CCSR_CATEGORY_Description'],inplace=True)

In [7]:
AHRQ_CCSR_df.drop_duplicates(inplace=True)
binary_columns = pd.get_dummies(AHRQ_CCSR_df['AHRQ_CCSR_CATEGORY'])

In [8]:
AHRQ_CCSR_df = pd.concat([AHRQ_CCSR_df, binary_columns], axis=1)
AHRQ_CCSR_df.drop(columns=['AHRQ_CCSR_CATEGORY'],inplace=True)
AHRQ_CCSR_df.drop_duplicates(inplace=True)

In [9]:
AHRQ_CCSR_df.to_gbq(
    destination_table='Fatemeh_db.Antimicrobial_Susceptibility_Comorbidities',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_14358/594233918.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  AHRQ_CCSR_df.to_gbq(


GenericGBQException: Reason: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/som-nero-phi-jonc101/datasets/Fatemeh_db/tables?prettyPrint=false: Invalid field name "Abdominal_pain_and_other_digestive/abdomen_signs_and_symptoms". Fields must contain the allowed characters, and be at most 300 characters long. For allowed characters, please refer to https://cloud.google.com/bigquery/docs/schemas#column_names

In [10]:
# Alixhuster comobidity 

In [ ]:
!git clone git@github.com:yubin-park/hcuppy.git

Cloning into 'hcuppy'...
The authenticity of host 'github.com (140.82.116.4)' can't be established.
ED25519 key fingerprint is SHA256:+DiY3wvvV6TuJJhbpZisF/zLDA0zPMSvHdkr4UvCOqU.
This key is not known by any other names.
Are you sure you want to continue connecting (yes/no/[fingerprint])? 

In [11]:
!pip install comorbidipy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
 df = pd.DataFrame({'id': [1,2,1,2,1], 
                    'age':[60,40,60,40,60],
                    'code': ['C25','C79','K27','G820','M32']})

In [1]:
import random
import unittest
from itertools import cycle

import pandas as pd
from comorbidipy import comorbidity, mapping

ccs = comorbidity(df, icd="icd10", score="charlson", variant="shmi", weighting="shmi")

ImportError: cannot import name 'SettingWithCopyWarning' from 'pandas.core.common' (/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/pandas/core/common.py)

In [16]:
!pip install --upgrade pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 26.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.0
    Uninstalling pandas-2.2.0:
      Successfully uninstalled pandas-2.2.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install pandas

  Using cached pandas-2.2.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (19 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_10_9_x86_64.whl (12.6 MB)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
